In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Path to the Shakespeare text file on Google Drive
file_path = '/content/drive/My Drive/shakespeare.txt'

# Reading the text file
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read().splitlines()


Mounted at /content/drive


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Initialize and fit tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
vocab_size = len(tokenizer.word_index) + 1

# Create sequences
input_sequence = []
for line in text:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequence.append(n_gram_sequence)

# Padding sequences to ensure all are the same length
input_sequence = pad_sequences(input_sequence, padding='pre')
max_len = input_sequence.shape[1]  # Correctly calculating max_len here


In [4]:
vocab_size = len(tokenizer.word_index) + 1  # Include +1 for zero padding


In [5]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Bidirectional

def build_generator(vocab_size, max_len):
    model = Sequential([
        Embedding(vocab_size, 100, input_length=max_len-1),
        Bidirectional(LSTM(128, return_sequences=True)),
        Dropout(0.2),
        LSTM(128),
        Dense(vocab_size, activation='softmax')
    ])
    return model

def build_discriminator(vocab_size, max_len):
    model = Sequential([
        Embedding(vocab_size, 100, input_length=max_len),
        Bidirectional(LSTM(128, return_sequences=True)),
        Dropout(0.2),
        LSTM(128),
        Dense(1, activation='sigmoid')
    ])
    return model


In [6]:
# Now use these variables to build and compile your models
generator = build_generator(vocab_size, max_len)

discriminator = build_discriminator(vocab_size, max_len)

from tensorflow.keras.optimizers import Adam

# Compile the discriminator
discriminator.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])

# Setup and compile the GAN
discriminator.trainable = False
gan_input = Input(shape=(max_len-1,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))


In [7]:
# noooooooooooooooo
# GAN Model where the discriminator's weights are frozen during generator training
discriminator.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])
discriminator.trainable = False
gan_input = Input(shape=(max_len-1,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), loss='binary_crossentropy')


In [8]:
def train_gan(epochs=200, batch_size=32):
    for epoch in range(epochs):
        idx = np.random.randint(0, input_sequence.shape[0], batch_size)
        real_seqs = input_sequence[idx]

        noise = np.random.normal(0, 1, (batch_size, max_len-1))
        fake_seqs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_seqs, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_seqs, np.zeros((batch_size, 1)))

        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        print(f"Epoch {epoch+1}/{epochs}, Discriminator Loss: {(d_loss_real + d_loss_fake) / 2}, Generator Loss: {g_loss}")


In [10]:
#### prev
def generate_text(seed_text, next_words=100):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')
        predicted_probs = generator.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs, axis=-1)[0]
        predicted_word = tokenizer.index_word.get(predicted_index, 'unknown')
        seed_text += " " + predicted_word
    return seed_text

# Example seed text
generated_text = generate_text("adorned in shades of red")
print(generated_text)


adorned in shades of red send shake shake slide expressed teeth holds bevel fury directly garments' try hardest just just air hated hated plot hope repent fiery fiery fiery fiery homage infant's infant's guard guard guard tall tall twofold twofold wake wake wrecked wrecked wrecked beguile beguile harsh answer undivided presence cherubins sullied known known often but but but but bier sickle's reckoning flown kills flown worthy injury injury offender's resting imprisoned livery sullen looked excess disgraced vice bitter bitter nothing nothing sunset bosom compile ere ere title fickle fickle decay jade short ever ever hindmost substance substance loss loss watchman chips night sickle's mortality


In [12]:
#EXP

import numpy as np

def sample(preds, temperature=1.0):
    """ Helper function to sample an index from a probability array. """
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-10) / temperature  # Adding a small number to avoid log(0)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(seed_text, next_words=100, temperature=0.8):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')
        predicted_probs = generator.predict(token_list, verbose=0)[0]
        next_index = sample(predicted_probs, temperature)  # Use temperature
        next_word = tokenizer.index_word.get(next_index, 'unknown')
        seed_text += " " + next_word
    return seed_text

# Generate text with a bit of randomness
generated_text = generate_text("adorned in shades of red", next_words=50, temperature=0.8)
print(generated_text)


adorned in shades of red adding bond buy raven any assured damasked oppression altered pluck philomel man's drown going left 'had unto fate weakness an pursuing judgement eat didst tempteth wolf able creation attaint birth ruined guides thence receive foregone surety prefiguring lords' wardrobe judgment simple shun threw whole fearful hated utmost untrimmed dross impute


In [13]:
#EXPPP
def generate_sonnet_line(initial_words):
    return generate_text(initial_words, next_words=10, temperature=0.8)  # Generates one line

def create_sonnet():
    lines = [generate_sonnet_line("The") for _ in range(14)]
    sonnet_structure = "\n".join([
        lines[0], lines[1], lines[4], lines[5],  # ABAB
        lines[2], lines[3], lines[6], lines[7],  # CDCD
        lines[8], lines[9], lines[12], lines[13],  # EFEF
        lines[10], lines[11]  # GG
    ])
    return sonnet_structure

# Example usage to generate a sonnet
generated_sonnet = create_sonnet()
print(generated_sonnet)



The stirred dare immured without carry sap snow possessed stamp life's
The perusal mend suffers sadly slave heretic tempests adverse keep burthens
The tend dry dian's scorned less belied o beside excuse heaven's
The shall bosom hang car misplaced sight privilege stays sought splendour
The register seals clay vows advocate sport renew tells flatter esteem
The tables prescriptions groan stealing colour confounding sufferance moiety semblance clear
The touch sweet'st tears married invention miscalled beautiful days garments' mak'st
The hounds bounteous directed anger stole destroys meetness belied even freedom
The together veil blazon flesh i'll pleased compound another's proof becomes
The saved expiate accents resort jollity salve tells helen's erst bitterness
The sullen blind present strengthened statues fits valley torment shall feed'st
The stewards frowns gravity again closet exceed oppressed sit subject small
The respects augurs elsewhere o'erworn fed awake hidden intelligence offenc

In [ ]:
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, sent_tokenize

# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

def calculate_sentiment(text):
    # Calculate sentiment scores
    sentiment_scores = sia.polarity_scores(text)
    return sentiment_scores

def calculate_lexical_diversity(text):
    # Calculate lexical diversity
    words = word_tokenize(text)
    unique_words = set(words)
    lexical_diversity = len(unique_words) / len(words)
    return lexical_diversity

def calculate_average_word_length(text):
    # Calculate average word length
    words = word_tokenize(text)
    total_chars = sum(len(word) for word in words)
    average_word_length = total_chars / len(words)
    return average_word_length

def calculate_coherence(text):
    # Calculate coherence (placeholder implementation)
    coherence_score = 0.0  # Placeholder, needs implementation
    return coherence_score

def check_iambic_pentameter(text):
    # Check iambic pentameter adherence (placeholder implementation)
    iambic_pentameter_adherence = False  # Placeholder, needs implementation
    return iambic_pentameter_adherence

def check_format(text):
    # Check format adherence (placeholder implementation)
    format_adherence = False  # Placeholder, needs implementation
    return format_adherence

# Example usage
generated_text = "For having traffic with thy self alone"
sentiment_scores = calculate_sentiment(generated_text)
lexical_diversity = calculate_lexical_diversity(generated_text)
average_word_length = calculate_average_word_length(generated_text)
coherence_score = calculate_coherence(generated_text)
iambic_pentameter_adherence = check_iambic_pentameter(generated_text)
format_adherence = check_format(generated_text)

# Print results
print("Sentiment Scores:", sentiment_scores)
print("Lexical Diversity:", lexical_diversity)
print("Average Word Length:", average_word_length)
print("Coherence Score:", coherence_score)
print("Iambic Pentameter Adherence:", iambic_pentameter_adherence)
print("Format Adherence:", format_adherence)


Sentiment Scores: {'neg': 0.25, 'neu': 0.75, 'pos': 0.0, 'compound': -0.25}
Lexical Diversity: 1.0
Average Word Length: 4.571428571428571
Coherence Score: 0.0
Iambic Pentameter Adherence: False
Format Adherence: False


In [ ]:
import matplotlib.pyplot as plt

# Plotting the losses over epochs
plt.plot(history.history['loss'])
plt.title('Model Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

NameError: name 'history' is not defined